In [2]:
import torch
import numpy as np

## 简单的例子
t = a + b

In [25]:
a = torch.full((3,3),1,dtype=torch.float32,requires_grad =True) # required_grad=True 开启即可反向传播求梯度
b = torch.full((3,3),2,dtype=torch.float32)
b.requires_grad =True
a,b

(tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]], requires_grad=True),
 tensor([[2., 2., 2.],
         [2., 2., 2.],
         [2., 2., 2.]], requires_grad=True))

In [26]:
# 运算
t = a + b
y = t.sum()
t.requires_grad

True

In [27]:
# 反向传播
y.backward()

In [31]:
# 获得梯度值
a.grad, b.grad, t.grad

C:\Users\gchen\AppData\Local\Temp\ipykernel_12856\1000682177.py:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  a.grad, b.grad, t.grad


(tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]),
 None)

## 例子
t = w*a + b  

a = [[1,1],[1,1]]  
b = [[2,2],[2,2]]  
w = [[2,2],[2,2]]  

In [50]:
a = torch.full((2,2),1,dtype=torch.float32,requires_grad =True) # required_grad=True 开启即可反向传播求梯度
b = torch.full((2,2),2,dtype=torch.float32,requires_grad =True)
w = torch.full((2,2),2,dtype=torch.float32,requires_grad =True)

In [56]:
y = w * a 
t = y + b
z = t.sum()

z.backward()

print(a.is_leaf, b.is_leaf, w.is_leaf, y.is_leaf, t.is_leaf)  # 判断是否为叶子节点
a.grad, b.grad, w.grad, y.grad, t.grad

True True True False False


C:\Users\gchen\AppData\Local\Temp\ipykernel_12856\1853962753.py:8: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  a.grad, b.grad, w.grad, y.grad, t.grad


(tensor([[10., 10.],
         [10., 10.]]),
 tensor([[5., 5.],
         [5., 5.]]),
 tensor([[5., 5.],
         [5., 5.]]),
 None,
 None)